In [1]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os

## 1. Escala de grises

In [3]:
def graycolor(img):
    image = cv2.imread(img)
    cv_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    return cv_gray

### Mujeres

In [9]:
f_carpetas = os.listdir('faces/female')

In [42]:
os.mkdir("EG_F")

In [43]:
path = "EG_F/"
for carp in f_carpetas:
    sub = os.listdir('faces/female/'+carp)
    for img in sub:
        file = 'faces/female/'+carp+"/"+img
        try:
            gray = graycolor(file)
            cv2.imwrite(os.path.join(path , img), gray)
            cv2.waitKey(0)
        except:
            continue

In [45]:
len(os.listdir("EG_F"))

399

### Hombres

In [4]:
m_carpetas = os.listdir('faces/male')
len(m_carpetas)

113

In [51]:
aleatorio = []
while len(aleatorio) <20:
    num = np.random.randint(113)
    if num in aleatorio:
        continue
    else:
        aleatorio.append(num)

In [52]:
len(aleatorio)

20

In [53]:
os.mkdir("EG_M")

In [54]:
path = "EG_M/"
for i in aleatorio:
    carp = 'faces/male/'+ m_carpetas[i]
    sub = os.listdir(carp)
    for img in sub:
        file = carp+"/"+img
        try:
            gray = graycolor(file)
            cv2.imwrite(os.path.join(path , img), gray)
            cv2.waitKey(0)
            #print(file)
        except:
            continue

In [55]:
len(os.listdir("EG_M"))

400

## 2. Extraer Vectores Caracteristicos

**accediendo a carpeta de trabajo**

In [2]:
os.listdir("../databases")

['EG_F', 'EG_M', 'Hog', 'faces']

In [3]:
os.chdir("../databases"), os.listdir()

(None, ['EG_F', 'EG_M', 'Hog', 'faces'])

### 2.1 Concatenar

En un vector para M y F

In [7]:
def extraeVector(image):
    img = cv2.imread(image)
    cv_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return list(cv_gray.ravel())

#### Mujeses

In [4]:
mujeres = os.listdir("EG_F")
len(mujeres)

399

In [9]:
female = []
for image in mujeres:
    vec = extraeVector('EG_F/'+image)
    female.append(vec)

In [10]:
female = np.array(female)
female.shape

(399, 36000)

#### Hoombres

In [5]:
hombres = os.listdir("EG_M")[:-1]
len(hombres)

399

In [12]:
male = []
for image in hombres:
    vec = extraeVector('EG_M/'+image)
    male.append(vec)

In [13]:
male = np.array(male)
male.shape

(399, 36000)

#### 2.1.1 PCA

In [14]:
from sklearn.decomposition import PCA

In [15]:
#MUJERES
pca = PCA(n_components=10)
PCAFemale = pca.fit_transform(female)

In [16]:
PCAFemale = pd.DataFrame(data=PCAFemale,
                   columns=["comp1","comp2","comp3","comp4","comp5",
                            "comp6","comp7","comp8","comp9","comp10"])
PCAFemale.head()

,comp1,comp2,comp3,comp4,comp5,comp6,comp7,comp8,comp9,comp10
0,3332.658797,-1571.427508,569.498921,-875.028444,-442.384966,-2619.423838,-344.317196,-1286.320936,60.252548,2003.012736
1,3375.603024,-2467.036414,-381.361538,6.202025,2807.441617,-598.126956,221.097276,-1047.190558,140.946255,1367.767043
2,-6429.357040,-3630.252199,-1362.103595,-354.267715,2386.927032,128.379549,884.071526,-2152.868232,-509.951895,516.006082
3,7957.378873,-1249.391673,-566.794251,-761.488113,1299.307416,-1110.461565,528.848087,-68.020410,1822.671284,864.210518
4,1624.824966,-1779.552152,-350.769143,-776.591574,-1361.591970,-1262.443450,-1309.651303,-772.186652,-345.804005,477.312084


In [17]:
#HOMBRES
pca = PCA(n_components=10)
PCAMale = pca.fit_transform(male)

In [18]:
PCAMale = pd.DataFrame(data=PCAMale,
                   columns=["comp1","comp2","comp3","comp4","comp5",
                            "comp6","comp7","comp8","comp9","comp10"])
PCAMale.head()

,comp1,comp2,comp3,comp4,comp5,comp6,comp7,comp8,comp9,comp10
0,2925.290478,420.238903,-2350.792841,-361.905468,-501.864950,1119.883543,620.995331,-237.479863,-620.765740,-55.996889
1,-1093.303275,-4949.874867,-3578.779879,1498.319461,-1242.512777,487.686908,-833.933390,839.994620,-771.451292,1960.352211
2,-2244.867709,3405.273167,941.848473,2645.450353,5472.047733,-1918.305834,2971.247879,985.382512,-1172.523848,1545.641532
3,-640.893722,-3573.628754,-593.127809,-3279.369843,220.802343,1385.721606,264.911401,669.443868,-2585.494080,427.768144
4,-2968.293045,3667.168584,2038.051202,-3445.821411,895.148996,4297.164550,2108.220282,-3245.916283,715.374229,60.445494


### 2.2 HOG

En un vector para M y F

In [6]:
def getHog(img):
    hog = cv2.HOGDescriptor()
    img = cv2.imread(img)
    cv_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    fv = hog.compute(cv_gray)
    return fv

**MUJERES**

In [7]:
fem_HOG = []
for image in mujeres:
    vec = getHog('EG_F/'+image)
    fem_HOG.append(vec)

In [8]:
fem_HOG = np.array(fem_HOG)[:,:,0]
fem_HOG.shape

(399, 567000)

In [10]:
fem_HOG[0:2,:]

array([[0.05096107, 0.10079551, 0.09052139, ..., 0.31633756, 0.31633756,
        0.05407228],
       [0.1692165 , 0.01800744, 0.04932511, ..., 0.19001985, 0.32638904,
        0.26495665]], dtype=float32)

**HOMBRES**

In [11]:
male_HOG = []
for image in hombres:
    vec = getHog('EG_M/'+image)
    male_HOG.append(vec)

In [12]:
male_HOG = np.array(male_HOG)[:,:,0]
male_HOG.shape

(399, 567000)

**Guardar vectores**

In [13]:
os.mkdir("FV_HOG")

In [15]:
print("tipo: ",type(fem_HOG), type(male_HOG))

tipo:  <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [18]:
np.save("FV_HOG/fem_HOG.npy", fem_HOG)
np.save("FV_HOG/male_HOG.npy", male_HOG)

### 2.3 SIFT

En carpetas

In [31]:
def getSift(img):
    sift = cv2.xfeatures2d.SIFT_create()
    img = cv2.imread(img)
    cv_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    kp, des = sift.detectAndCompute(cv_gray,None)
    return des

In [32]:
os.mkdir("FV_SIFT")
os.mkdir("FV_SIFT/female")
os.mkdir("FV_SIFT/male")

**MUJERES**

In [61]:
getSift("EG_F/"+mujeres[1]).shape

(96, 128)

In [53]:
".".join(mujeres[3].split(".")[:-1])

'sbains.15'

In [62]:
for image in mujeres:
    vec = getSift('EG_F/'+image)
    name = ".".join(image.split(".")[:-1])
    np.save("FV_SIFT/female/"+name+".npy", vec)#GUARDA

In [73]:
len(os.listdir("FV_SIFT/female"))

399

**HOMBRES**

In [63]:
for image in hombres:
    vec = getSift('EG_M/'+image)
    name = ".".join(image.split(".")[:-1])
    np.save("FV_SIFT/male/"+name+".npy", vec)#GUARDA

In [74]:
len(os.listdir("FV_SIFT/male"))

399

### 2.4 SURF

En carpetas

In [68]:
def getSurf(img):
    surf = cv2.xfeatures2d_SURF.create(400)
    img = cv2.imread(img)
    cv_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    kp, des = surf.detectAndCompute(cv_gray,None)
    return des

In [66]:
os.mkdir("FV_SURF")
os.mkdir("FV_SURF/female")
os.mkdir("FV_SURF/male")

**MUJERES**

In [69]:
for image in mujeres:
    vec = getSurf('EG_F/'+image)
    name = ".".join(image.split(".")[:-1])
    np.save("FV_SURF/female/"+name+".npy", vec)#GUARDA

In [71]:
len(os.listdir("FV_SURF/female"))

399

**HOMBRES**

In [70]:
for image in hombres:
    vec = getSurf('EG_M/'+image)
    name = ".".join(image.split(".")[:-1])
    np.save("FV_SURF/male/"+name+".npy", vec)#GUARDA

In [72]:
len(os.listdir("FV_SURF/male"))

399